In [ ]:
import random
import re

import pandas as pd
import numpy as np
import joblib

In [ ]:
def to_snake_case(item):
    """Convert input string to 'snake_case' format."""
    regex = r'(?<!^)(?=[A-Z])'
    return re.sub(regex, '_', item).lower()


In [ ]:
data_raw = pd.read_csv('FimaNfipClaims.csv', low_memory=False)

In [ ]:
data_interim = data_raw.copy()
data_interim.columns = [to_snake_case(item) for item in data_raw.columns]


In [ ]:
data_interim = (
    data_interim
    .fillna({
        'amount_paid_on_increased_cost_of_compliance_claim': 0,
        'amount_paid_on_building_claim': 0,
        'amount_paid_on_contents_claim': 0,
    })
    .assign(
        total_insurance_value=lambda x: (
            x['total_building_insurance_coverage'] +
            x['total_contents_insurance_coverage']
        ),
        total_loss=lambda x: (
            x['amount_paid_on_building_claim'] +
            x['amount_paid_on_contents_claim']
        ),
        loss_ratio_building=lambda x: (
            x['amount_paid_on_building_claim'] /
            x['total_building_insurance_coverage']
        ),
        loss_ratio_content=lambda x: (
            x['amount_paid_on_contents_claim'] /
            x['total_contents_insurance_coverage']
        ),
    )
    .assign(total_loss_ratio=lambda x: x['total_loss'] / x['total_insurance_value'])
)

In [ ]:
# Keep only year for year built of the building

default_date = '1700-01-01T00:00:00.000Z'
data_interim['original_construction_date'] = (
    data_interim
    .loc[:, 'original_construction_date']
    .fillna(default_date)
    .astype(str)
    .replace(regex=['1492-10-12T00:00:00.000Z', '0001-01-01T00:00:00.000Z'], value=default_date)
    .astype('datetime64[ns]')
    .dt.year
)


In [ ]:
data_interim['rated_flood_zone'] = (
    data_interim
    ['rated_flood_zone']
    .fillna('UNK')
    .replace({
        r'((?!A99)A[0-9]+)': 'AE',
        r'(AH[\w]+)': 'AH',
        r'(AO[\w]+)': 'AO',
        r'(V[0-9]+)': 'VE',
        'B': 'X',
        'C': 'X',
        'D': 'X',
    }, regex=True)
)


In [ ]:
data_interim['reported_zip_code'] = (
    data_interim
    ['reported_zip_code']
    .fillna(0)
    .astype(int)
)


In [ ]:
data_interim['elevation_difference'] = (
    data_interim
    .loc[:, ['lowest_floor_elevation', 'base_flood_elevation']]
    .fillna(0.0)
    .astype('float64')
    .assign(elevation_difference=lambda x: (x['lowest_floor_elevation'] - x['base_flood_elevation']).round())
    .loc[:, 'elevation_difference']
)


In [ ]:
data_cleaned = (
    data_interim
    .drop([
        'as_of_date',
        'county_code',
        'census_tract',
        'reported_city',
        'date_of_loss',
        'elevation_certificate_indicator',
        'lowest_adjacent_grade',
        'lowest_floor_elevation',
        'base_flood_elevation',
        # 'original_construction_date',
        'original_n_b_date',
        'amount_paid_on_increased_cost_of_compliance_claim',
        'rate_method',
        'small_business_indicator_building',
        'state',
        # 'reported_zip_code',
        'primary_residence_indicator',
    ], axis=1)
    [lambda x: (
        (x['latitude'].notna())
        & (x['longitude'].notna())
        & (x['number_of_floors_in_the_insured_building'].notna())
        & ((x['elevation_difference'] <= 30) & (x['elevation_difference'] >= -30))
        & (x['original_construction_date'] >= 1800)
        & (x['non_profit_indicator'] != '0')
        & (x['reported_zip_code'] > 100)
        & (x['total_insurance_value'] != 0)
        & (x['obstruction_type'] != '*')
        & (x['rated_flood_zone'] != 'UNK')
    )]
    .fillna({
        'obstruction_type': 'UNK',
        'agriculture_structure_indicator': 'N',
        'basement_enclosure_crawlspace_type': 0,
        'condominium_coverage_type_code': 'N',
        'policy_count': 1,
        'crs_classification_code': 0,
        'elevated_building_indicator': 'UNK',
        'rated_flood_zone': 'UNK',
        'house_worship': 'N',
        'location_of_contents': 'UNK',
        'non_profit_indicator': 'N',
        'occupancy_type': 1.0,
        'post_f_i_r_m_construction_indicator': 'UNK',
    })
    # .astype({'obstruction_type': 'str'})
    .reset_index(drop=True)
)


In [ ]:
data_cleaned.columns.tolist()

In [ ]:
data_cleaned.to_csv('FEMA_Data_Cleaned_Regression_V2.csv', index=False)
